In [1]:
import numpy as np
import pandas as pd
# import dcMinMaxFunctions as dc
# import dcor
from scipy.misc import derivative
from sklearn.model_selection import train_test_split
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
import wandb
from opacus import PrivacyEngine
from cov_help import *
import time
import argparse



In [2]:
X,Y = cov_data_loader("data/covtype.csv",norm=1)

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
batch_size = 4096
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train,Y_train), batch_size=batch_size,
                                        shuffle=True, num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test,Y_test), batch_size=batch_size,
                                        shuffle=True, num_workers=2)

#write code to append Xemb and Xemb_rest

criterion = nn.CrossEntropyLoss()

model = nn.Sequential(
        nn.Linear(54, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 7),
        nn.Softmax(dim=1)
    )

In [4]:

import json

# do
import math
import sys

# from pvc import Dataset
from typing import List, Optional, Tuple

import numpy as np

# from stl.lightning.loggers import ManifoldTensorBoardLogger
# from stl.lightning.callbacks.all import ModelCheckpoint
import pytorch_lightning as pl
from sklearn.decomposition import PCA

# do
from torch.nn.utils import clip_grad_norm_

# from opacus import PrivacyEngine
# from opacus.data_loader import DPDataLoader
# from opacus.validators.module_validator import ModuleValidator
from tqdm import tqdm

In [5]:

# TODO: clip norm


class TreeNode:
    def __init__(self, depth: int, value: float, efficient: bool):
        """
        implements the nodes of the tree. if the tree is efficient we will weigh
        the nodes by the weight method. adapted from opacus privacy engine.
        """
        self.depth = depth
        self.value = value
        self.efficient = efficient

    def update_value(self, indx, new_value):
        self.value[indx] += new_value

    def get_value(self, indx):
        """as indicated by the efficient implementation of tree aggregation
        we reweight it by 1 / (2 - 2^(-depth)) where depth starts from the root.
        ex. depth(root) = 0
        """
        if self.efficient:
            # print('dd', self.depth)
            # print('efficient test', float((1.0 / (2 - math.pow(2, -self.depth))) ** 0.5) * self.value)
            return ((1.0 / (2 - math.pow(2, -self.depth))) ** 0.5) * self.value[indx]
        else:
            return 1.0 * self.value[indx]


class TreeAggregation:
    """
    Correlated noise added to gradient sums from a private binary tree
    Efficient Implementation Paper (reduces variance in sum of gaussian noise values):
        Efficient Use of Differentially Private Binary Trees
        https://privacytools.seas.harvard.edu/files/privacytools/files/honaker.pdf
    Vanilla Implementation:
        Differential Privacy Under Continual Observation
        http://www.wisdom.weizmann.ac.il/~naor/PAPERS/continual_observation.pdf

    Args:
        std = standard deviation of gaussian noise
        efficient = decides which kind of tree aggregation to be used
        seed = added for reproducibility
    """

    def __init__(
        self,
        grad_sizes,
        std: float = 1.0,
        efficient: bool = False,
        seed: int = 0,
        mode: str = "prod",  # alternatives are prod/test
    ):
        self.efficient = efficient
        self.std = std
        self.seed = seed
        self.mode = mode
        self.grad_sizes = grad_sizes

        self.step = 1
        self.depth = 0  # TODO: increase depth appropriately
        self.max_nodes = 2 ** (self.depth)
        self.tree = []

        # reproducible experiments
        self.generator = torch.Generator()
        self.generator.manual_seed(self.seed)

    def add_to_tree_helper(
        self,
        gradient,
        noise_to_add,
        binary_repr_step,
        step,
        max_nodes,
        add_gradient,
    ):
        """
        adds noise to each node on the path from the root node to the leaf node.
        Also, returns correlated noise for this specific path to be added to the FTRL optimizer.

        Args:
            noise_to_add = torch Tensor (gaussian noise) to be added
            binary_repr_step = convert current step value to binary representation
            step = number of times this function has been called (correlated with maximum nodes in the system)
            max_nodes = maximum possible nodes in the tree (i.e. counted after assuming a complete binary tree,
                where each node except the leaf node have exactly two childen)

        Returns:
            correlated noise determined from the current tree structure & step number
        """
        if len(self.tree) == 0:
            # print('Root Added')
            noise_sum = [torch.zeros(grad_size) for grad_size in self.grad_sizes]
            self.tree.append(
                TreeNode(depth=self.depth, value=noise_to_add, efficient=self.efficient)
            )
            # print(len(self.tree), gradient, len(gradient))
            if add_gradient:
                for indx in range(len(self.grad_sizes)):
                    self.tree[0].update_value(indx, gradient[indx])

            for indx in range(len(self.grad_sizes)):
                noise_sum[indx] += self.tree[0].get_value(indx)
            return noise_sum

        # node_indx = 0
        binary_step_indx = 1

        # append noise & init noise_sum
        self.tree.append(
            TreeNode(depth=self.depth, value=noise_to_add, efficient=self.efficient)
        )
        noise_sum = [torch.zeros(grad_size) for grad_size in self.grad_sizes]

        # for indx in range(len(self.grad_sizes)):
        #    self.tree[step].update_value(indx, gradient[indx])

        # get back value
        if binary_repr_step[0] == "1":  # test if left node
            for indx in range(len(self.grad_sizes)):
                noise_sum[indx] += self.tree[step].get_value(indx)

        steps = [step]
        while step >= 0 and binary_step_indx < len(binary_repr_step):
            left_indx = int((step - 1) / 2)
            if step % 2 == 1:  # is left child
                step = int((step - 1) / 2)
            else:  # is right node
                step = int((step - 2) / 2)
            steps.append(step)
            if add_gradient:
                for indx in range(len(self.grad_sizes)):
                    self.tree[step].update_value(indx, gradient[indx])

            # get value
            if binary_repr_step[binary_step_indx] == "1" and left_indx < len(self.tree):
                for indx in range(len(self.grad_sizes)):
                    noise_sum[indx] += self.tree[left_indx].get_value(indx)

            binary_step_indx += 1

        # print(steps)
        return noise_sum

    def print_tree(self):
        print("------------------------Tree Starts------------------------")
        for indx in range(len(self.tree)):
            print(
                "(" + str(indx + 1) + "," + str(self.tree[indx].value[0].item()) + ")",
                end=" ",
            )
        print("\n------------------------Tree Ends------------------------")

    def add_to_tree_and_get_sum(
        self,
        gradient: torch.Tensor,
        test_noise: Optional[torch.Tensor] = None,
        add_gradient: bool = False,
    ) -> None:
        """
        add the new value to each node along the path

        Args:
            test_noise = torch.normal(mean=0.0, std=self.std, generator=torch.Generator())

        Returns:
            None
        """
        if self.mode == "prod":
            noise_to_add = []
            for indx in range(len(self.grad_sizes)):
                # print('size', self.grad_sizes[indx])
                noise_to_add.append(
                    torch.normal(
                        mean=0.0,
                        std=self.std,
                        size=self.grad_sizes[indx],
                        # generator=self.generator
                    )
                )
        elif self.mode == "test":
            noise_to_add = test_noise

        noise_sum = self.add_to_tree_helper(
            gradient,
            noise_to_add,
            np.binary_repr(self.step)[::-1],
            self.step - 1,
            self.max_nodes + 1,
            add_gradient,
        )

        self.step += 1
        # print('metric', self.step, self.max_nodes, self.depth)
        if self.step > self.max_nodes:
            self.depth += 1
            self.max_nodes += 2 ** (self.depth)

        return noise_sum


from typing import List

# do
import torch
from torch.optim.optimizer import Optimizer, required


class FTRLM(Optimizer):
    """
    implements FTRL optimizer to be used with Opacus or alternative noise generation
    mechanism with (optional) momentum

    DP-FTRL (private) paper: https://arxiv.org/abs/2103.00039
    FTRL (non-private) paper: https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/41159.pdf
    """

    def __init__(
        self,
        params,  # TODO: add tree aggregator
        model_param_sizes: List[torch.Size] = [(1,)],
        lr: float = 0.0,
        momentum: float = 0.0,
        dampening: float = 0.0,
        nesterov: bool = False,
        noise_std: float = 0.0,
        max_grad_norm: float = 100.0,
        seed: int = 0,
        efficient: bool = False,
        is_PCA_enabled: bool = False,
        PCA_variance: float = 0.95,
    ):
        """
        Args:
            params = parameters for torch optimizer
            lr = learning rate
            momentum = # TODO: add implementation
            nesterov = # TODO: add implementation
            dampening = # # TODO: add implementation
            noise_std = amount of noise to be added to teh
        """

        # sanity checks for parameters
        if lr <= 0.0:
            raise ValueError("Invalid Learning Rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid Momentum Parameter: {}".format(momentum))
        if isinstance(max_grad_norm, float) and max_grad_norm <= 0:
            raise ValueError(
                "max_grad_norm = {} is not a valid value. Please provide a float > 0.".format(
                    max_grad_norm
                )
            )

        self.max_grad_norm = max_grad_norm
        self.seed = seed
        self.noise_std = noise_std
        self.efficient = efficient
        self.is_PCA_enabled = is_PCA_enabled
        self.PCA_variance = PCA_variance

        self.tree_aggregator = TreeAggregation(
            grad_sizes=model_param_sizes,
            std=self.noise_std,
            efficient=self.efficient,
            seed=self.seed,
        )

        # TODO: add weight decay
        # if not 0.0 < weight_decay:
        #    raise ValueError("Invalid Weight Decay Parameter: {}".format(weight_decay))

        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")

        defaults = dict(
            lr=lr, dampening=dampening, momentum=momentum, nesterov=nesterov
        )

        super().__init__(params, defaults)

    def clip_gradient(self, grad):  # TODO: add per-example clipping
        # print('Sanity Check', grad.shape)
        if torch.norm(grad) > self.max_grad_norm:
            return (grad * self.max_grad_norm) / float(torch.norm(grad))
        return grad

    def step(self, closure=None):
        """performs one step of the DP-FTRL algorithm.

        Args:
            closure (callable, optional): a closure that reevaluates
            the model and returns the loss
        """

        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        gradients = []
        noisy_gradients = self.tree_aggregator.add_to_tree_and_get_sum(
            gradients, add_gradient=False
        )
        for group in self.param_groups:
            momentum = group["momentum"]
            dampening = group["dampening"]
            nesterov = group["nesterov"]
            lr = group["lr"]
            for p, noise in zip(group["params"], noisy_gradients):
                if p.grad is None:
                    continue

                d_p = p.grad
                d_p = self.clip_gradient(d_p)
                param_state = self.state[p]
                if len(param_state) == 0:
                    param_state["grad_sum"] = torch.zeros_like(d_p)
                    param_state["momentum"] = torch.zeros_like(p)
                    param_state["initial_model"] = torch.zeros_like(p)
                    param_state["initial_model"].add_(p)  # only add initial model

                initial_model, grad_sum = (
                    param_state["initial_model"],
                    param_state["grad_sum"],
                )

                grad_sum.add_(d_p)
                # PCA
                # noisy_gradient = noisy_gradients[indx]
                """if self.is_PCA_enabled:
                    pca = PCA(self.PCA_variance)
                    dimension2 = 1
                    gradient_shape = noisy_gradient.shape
                    for i in range(len(noisy_gradient.shape)):
                        if i != 0:
                            dimension2 *= gradient_shape[i]
                    noisy_gradient_2D = noisy_gradient.reshape((gradient_shape[0], dimension2))
                    noisy_transformed_gradient = pca.fit_transform(noisy_gradient_2D)"""

                param_state["momentum"].mul_(momentum).add_(
                    grad_sum + noise, alpha=(1 - dampening)
                )

                # Nesterov
                if nesterov:
                    delta_w = (grad_sum + noise).add(
                        param_state["momentum"], alpha=momentum
                    )
                else:
                    delta_w = param_state["momentum"]

                # p = initial_model
                with torch.no_grad():
                    p.copy_(initial_model - delta_w * lr)

        return loss


# do
import unittest
from copy import deepcopy


class FTRLMTest(unittest.TestCase):
    # init
    def __init__(self):
        pass
        # add multiple tests

    # utils
    def change_lr(self, opt, new_lr):
        for group in opt.param_groups:
            group["lr"] = new_lr

    def get_lr(self, opt):
        for group in opt.param_groups:
            return group["lr"]

    def get_momentum(self, opt):
        for group in opt.param_groups:
            return group["momentum"]

    # tests
    def test_lr_deterministic(self):
        param = torch.tensor([0.0], requires_grad=True)
        param.grad = torch.tensor([1.0])
        self.optimizer = FTRLM([param], lr=1, noise_std=0.0, max_grad_norm=10.0)
        # self.optimizer.set_param_groups([param], lr=1)

        # output = 0 - 1.0 * 1.0 = -1.0
        self.assertAlmostEqual(self.get_lr(self.optimizer), 1.0)
        self.optimizer.step()  # 1st step
        print(param.item())
        self.assertAlmostEqual(param.item(), -1.0, delta=1e-5)

        # output = 0 - 1.5 * (1.0 + 1.0) = -3.0
        self.change_lr(self.optimizer, 1.5)  #
        self.assertAlmostEqual(self.get_lr(self.optimizer), 1.5)
        self.optimizer.step()
        print(param.item())
        self.assertAlmostEqual(param.item(), -3.0, delta=1e-5)

    def test_momentum(self):
        param = torch.zeros((784, 10), requires_grad=True)
        shapes = [param.shape]
        # shapes = [p.shape for p in param]
        print(shapes)
        param.grad = torch.ones_like(param)
        self.optimizer = FTRLM(
            [param],
            lr=0.1,
            model_param_sizes=shapes,
            noise_std=0.0,
            momentum=0.9,
            max_grad_norm=90.0,
        )

        self.assertAlmostEqual(self.get_lr(self.optimizer), 0.1)
        self.assertAlmostEqual(self.get_momentum(self.optimizer), 0.9)
        for epoch in range(2):
            self.optimizer.step()
        output = torch.flatten(param)
        print(output)
        self.assertTrue(
            torch.allclose(
                output,
                torch.Tensor(
                    [-0.29 * np.ones_like(p.detach().numpy()) for p in output]
                ),
            )
        )

    def test_ftrl_similar_sgd(self):  # without nosie should perform simlarly to SGD
        param_optimizer = torch.normal(
            mean=0.0, std=1.0, size=(784, 10), requires_grad=True
        )
        # param_optimizer = torch.zeros((784, 10))
        param_optimizer.grad = torch.normal(
            mean=0.0, std=1.0, size=(784, 10), requires_grad=True
        )
        # print('init grad', param_optimizer.grad)
        shapes = [param_optimizer.shape]
        param_sgd = deepcopy(param_optimizer)
        param_sgd.grad = deepcopy(param_optimizer.grad)
        self.optimizer = FTRLM(
            [param_optimizer],
            model_param_sizes=shapes,
            lr=0.01,
            noise_std=0.0,
            max_grad_norm=500.0,
        )
        self.sgd = torch.optim.SGD([param_sgd], lr=0.01)

        for epoch in range(3):
            self.optimizer.step()
            self.sgd.step()
        print((param_optimizer), (param_sgd))
        self.assertTrue(
            torch.allclose(torch.flatten(param_optimizer), torch.flatten(param_sgd))
        )

    def test_ftrl_similar_sgd_momentum(
        self,
    ):  # without nosie should perform simlarly to SGD
        param_optimizer = torch.normal(
            mean=0.0, std=1.0, size=(784, 10), requires_grad=True
        )
        # param_optimizer = torch.zeros((784, 10))
        param_optimizer.grad = torch.normal(
            mean=0.0, std=1.0, size=(784, 10), requires_grad=True
        )
        # print('init grad', param_optimizer.grad)
        shapes = [param_optimizer.shape]
        param_sgd = deepcopy(param_optimizer)
        param_sgd.grad = deepcopy(param_optimizer.grad)
        self.optimizer = FTRLM(
            [param_optimizer],
            model_param_sizes=shapes,
            lr=0.01,
            noise_std=0.0,
            max_grad_norm=500.0,
            momentum=0.9,
        )
        self.sgd = torch.optim.SGD([param_sgd], lr=0.01, momentum=0.9)

        for epoch in range(3):
            self.optimizer.step()
            self.sgd.step()
        print((param_optimizer), (param_sgd))
        self.assertTrue(
            torch.allclose(torch.flatten(param_optimizer), torch.flatten(param_sgd))
        )


test = FTRLMTest()
test.test_lr_deterministic()  # DONE
test.test_momentum()
test.test_ftrl_similar_sgd()
test.test_ftrl_similar_sgd_momentum()

####DATALOADERS####

####NET####


# do
import matplotlib.pyplot as plt


-1.0
-3.0
[torch.Size([784, 10])]
tensor([-0.2900, -0.2900, -0.2900,  ..., -0.2900, -0.2900, -0.2900],
       grad_fn=<ViewBackward0>)
tensor([[ 1.2115, -0.7429, -0.8727,  ..., -0.3220,  0.9429,  0.7288],
        [ 0.3995,  0.3587, -0.5243,  ...,  0.4212, -1.0353, -0.8312],
        [ 0.3912,  0.4951,  0.6812,  ...,  1.4518,  0.2833,  1.4999],
        ...,
        [ 1.2337, -0.5575,  1.5038,  ...,  0.2897, -0.6609, -0.8485],
        [-0.9495,  1.0158, -0.0466,  ..., -0.4603, -0.5817,  1.8481],
        [ 0.4128, -1.4541, -1.0309,  ..., -0.4845,  0.9611, -0.5176]],
       requires_grad=True) tensor([[ 1.2115, -0.7429, -0.8727,  ..., -0.3220,  0.9429,  0.7288],
        [ 0.3995,  0.3587, -0.5243,  ...,  0.4212, -1.0353, -0.8312],
        [ 0.3912,  0.4951,  0.6812,  ...,  1.4518,  0.2833,  1.4999],
        ...,
        [ 1.2337, -0.5575,  1.5038,  ...,  0.2897, -0.6609, -0.8485],
        [-0.9495,  1.0158, -0.0466,  ..., -0.4603, -0.5817,  1.8481],
        [ 0.4128, -1.4541, -1.0309,  ...,

In [6]:
model.to(torch.device("cpu"))

Sequential(
  (0): Linear(in_features=54, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=128, bias=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=64, bias=True)
  (9): ReLU()
  (10): Linear(in_features=64, out_features=64, bias=True)
  (11): ReLU()
  (12): Linear(in_features=64, out_features=7, bias=True)
  (13): Softmax(dim=1)
)

In [7]:
bs = 4096
noise_multiplier = 7.0
print("#############params###############")
param_list = []
param_shapes = [p.shape for p in model.parameters()]
for p in model.parameters():
    param_list.append(p)
print("#############params###############")        


optimizer_FTRLM = FTRLM( # noqa
        params=param_list, 
        model_param_sizes=param_shapes, 
        lr=0.01, 
        momentum=0.9, 
        nesterov=True, 
        noise_std=(noise_multiplier / bs),
        max_grad_norm=10.0,
        seed=0,
        efficient=True,
        )

#############params###############
#############params###############


In [8]:

from absl import app
import numpy as np
from typing import List
from collections import Counter
import json

def convert_gaussian_renyi_to_dp(sigma, delta, verbose=True):
    """
    Convert from RDP to DP for a Gaussian mechanism.
    :param sigma: the algorithm guarantees (alpha, alpha/(2*sigma^2))-RDP
    :param delta: target DP delta
    :param verbose: whether to print message
    :return: the DP epsilon
    """
    alphas = np.arange(1, 200, 0.1)[1:]
    epss = alphas / 2 / sigma**2 - (np.log(delta*(alphas - 1)) - alphas * np.log(1 - 1/alphas)) / (alphas - 1)
    idx = np.nanargmin(epss)
    if verbose and idx == len(alphas) - 1:
        print('The best alpha is the last one. Consider increasing the range of alpha.')
    eps = epss[idx]
    return eps


def get_total_sensitivity_sq_given_order(order):
    """
    Get the squared sensitivity for a given order of batches.
    Can be viewed as a general case for get_total_sensitivity_sq_same_order.
    This function is not used in the privacy computation, as we operated in the case where the
    data order is the same for every epoch.
    :param order: a list representing the order of the batches, e.g. [0,1,2,1] means we use batch indexed with 0,1,2,1.
                  -1 indicates virtual step.
    :return: squared sensitivity, squared_sensitivity with respect to all every batch
    """
    # get first layer as a list of counters
    layer = [Counter({node: 1}) for node in order]

    # sensitivity_sq[i] will record the total sensitivity wrt batch i
    sensitivity_sq_all = [0] * (max(order) + 1)

    # update sensitivity_sq with a given layer
    def update_sensitivity_sq(current_layer):
        for node in current_layer:
            for ss in node:
                if ss != -1:
                    sensitivity_sq_all[ss] += node[ss] ** 2

    update_sensitivity_sq(layer)  # get sensitivity for the first layer
    while len(layer) > 1:
        layer_new = []  # merge every two consecutive nodes to get the next layer
        length = len(layer)
        for i in range(0, length, 2):
            if i + 1 < length:
                layer_new.append(layer[i] + layer[i + 1])
        layer = layer_new
        update_sensitivity_sq(layer)
    return max(sensitivity_sq_all), sensitivity_sq_all


def get_total_sensitivity_sq_same_order(steps_per_epoch, epochs, extra_steps, mem_fn=None):
    """
    Get the squared sensitivty for a tree where we fix the order of batches for all epochs.
    :param steps_per_epoch: number of steps per epoch
    :param epochs: number of epochs in the tree
    :param extra_steps: number of virtual steps
    :param mem_fn: if set, will write result to the file
    :return: squared sensitivity, squared sensivity assuming no virtual steps,
             squared sensitivity with respect to every batch
    e.g. steps_per_epochs = 3 and epochs = 2, extra_steps = 2 means we have three batches b1, b2, b3,
    and train w/ [b1, b2, b3, b1, b2, b3, +, +] where + means the extra steps.
    We will enumerate through all nodes layer by layer in list "layer", and  compute the sensitivty
    with respect to every node in "sensitivity_sq".
    """
    # to record the result to save computation
    mem = json.load(open(mem_fn)) if mem_fn else {}
    key = f'{steps_per_epoch},{epochs},{extra_steps}'
    key_no_extra = f'{steps_per_epoch},{epochs},{0}'
    if key in mem and key_no_extra in mem:
        return mem[key], mem[key_no_extra], None

    # get first layer as a list of counters, the keys are batches (indexed with non-negative numbers), counts are
    # number of times the batch appears in the node
    layer = []
    for _ in range(epochs):
        layer += [Counter({ss: 1}) for ss in range(steps_per_epoch)]
    layer += [Counter({-1: 1}) for _ in range(extra_steps)]  # extra steps denoted as -1

    # sensitivity_sq[i] will record the total sensitivity wrt batch i
    sensitivity_sq_all = [0] * steps_per_epoch
    sensitivity_sq_all_no_extra = [0] * steps_per_epoch  # will also compute sensitivity without extra

    # update sensitivity_sq with a given layer
    def update_sensitivity_sq(current_layer):
        for node in current_layer:
            has_extra = -1 in node
            for ss in node:
                if ss != -1:
                    sensitivity_sq_all[ss] += node[ss] ** 2
                    if not has_extra:
                        sensitivity_sq_all_no_extra[ss] += node[ss] ** 2

    update_sensitivity_sq(layer)  # get sensitivity for the first layer
    while len(layer) > 1:
        layer_new = []  # merge every two consecutive nodes to get the next layer
        length = len(layer)
        for i in range(0, length, 2):
            if i + 1 < length:
                layer_new.append(layer[i] + layer[i + 1])
        del layer
        layer = layer_new
        update_sensitivity_sq(layer)

    # save to file
    if mem_fn:
        mem[key] = max(sensitivity_sq_all)
        mem[key_no_extra] = max(sensitivity_sq_all_no_extra)
        with open(mem_fn, 'w') as f:
            json.dump(mem, f, indent=4)
    return max(sensitivity_sq_all), max(sensitivity_sq_all_no_extra), sensitivity_sq_all


def compute_epsilon_tree_restart_rdp_same_order_extra(num_batches: int, epochs_between_restarts: List[int],
                                                      noise: float, tree_completion: bool = True,
                                                      mem_fn: str = None):
    """
    Compute the effective noise for DP-FTRL.
    :param num_batches: number of batches per epoch
    :param epochs_between_restarts: number of epochs between each restart, e.g. [2, 1] means epoch1, epoch2, restart, epoch3
    :param noise: noise multiplier for each step
    :param tree_completion: if true, use the tree completion trick which adds virtual steps to complete the binary tree
    :param mem_fn: if set, will write result to the file
    :return: the effective noise for DP-FTRL
    """
    if noise < 1e-20:
        return float('inf')

    mem = {}  # to record result to avoid computing the same setting twice
    sensitivity_sq = 0  # total sensitivity^2, which is the sum over all "intervals" between each restarting
    for i, epochs in enumerate(epochs_between_restarts):
        if epochs == 0:
            continue
        if tree_completion and i < len(epochs_between_restarts) - 1:
            # compute number of virtual steps
            extra_steps = 2 ** (num_batches * epochs - 1).bit_length() - num_batches * epochs
        else:
            extra_steps = 0
        key = (num_batches, epochs, extra_steps)
        mem[key] = mem.get(key,
                           get_total_sensitivity_sq_same_order(num_batches, epochs, extra_steps, mem_fn)[0])
        sensitivity_sq += mem[key]
    effective_sigma = noise / np.sqrt(sensitivity_sq)
    return effective_sigma


def compute_epsilon_tree(num_batches: int, epochs_between_restarts: List[int], noise: float, delta: float,
                         tree_completion: bool,
                         verbose=True, mem_fn=None):
    """
    Compute epsilon value for DP-FTRL.
    :param num_batches: number of batches per epoch
    :param epochs_between_restarts: number of epochs between each restart, e.g. [2, 1] means epoch1, epoch2, restart, epoch3
    :param noise: noise multiplier for each step
    :param delta: target DP delta
    :param tree_completion: if true, use the tree completion trick which adds virtual steps to complete the binary tree
    :param verbose: whether to print message
    :param mem_fn: if set, will write result to the file
    :return: the DP epsilon for DP-FTRL
    """

    if noise < 1e-20:
        return float('inf')

    effective_sigma = compute_epsilon_tree_restart_rdp_same_order_extra(num_batches, epochs_between_restarts, noise,
                                                                        tree_completion, mem_fn)
    eps = convert_gaussian_renyi_to_dp(effective_sigma, delta, verbose)
    return eps




In [9]:
pip install absl-py

Note: you may need to restart the kernel to use updated packages.


In [10]:
n = 400000
delta = 1e-5
batch = 4096
epochs = 100
restart_every = 1
noise = 1000.0
tree_completion = True

num_batches = n // batch
epochs_between_restarts = [restart_every] * (epochs // restart_every)

eps = compute_epsilon_tree(num_batches, epochs_between_restarts, noise, delta, tree_completion)

print(f'n={n}, batch={batch}, epochs={epochs} with restarting every {restart_every} epochs', f'noise={noise}, tree_completion={tree_completion}',f'gives ({eps:.2f}, {delta})-DP')

n=400000, batch=4096, epochs=100 with restarting every 1 epochs noise=1000.0, tree_completion=True gives (0.10, 1e-05)-DP


In [11]:
import wandb
wandb.init(project="covtype",name="FTRL")
num_epochs = 100
max_steps = 100000

05/18/2024 15:59:46:ERROR:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ponkshekaustubh11. Use `wandb login --relogin` to force relogin


In [12]:
train_emb(model, train_loader, criterion, optimizer_FTRLM, num_epochs=num_epochs,device=torch.device('cpu'),test_loader = test_loader,max_steps = max_steps)